# 🏋️ Bootcamp: Real-Time Hand Tracking for Sports Biomechanics

**Goal**: Introduce students to computer vision-based biomechanical analysis using hand tracking, and encourage creative sports-related applications.

------

##✅ Step 1: Introduction to Biomechanics in Sports

**Goal**: Understand how computer vision can be applied to human motion and sports analysis.

###Topics:
- What is biomechanics in sports?
- Real-world use cases: swing analysis in cricket, gait analysis for rehab, injury prevention
------

## Note: Please run this code on your local Jupyter. Google Colab doesn't connect with your webcam.


## ✅ Step 2: Setup & Install Prerequisites

**Goal**: Set up the coding environment for real-time camera capture and tracking.

###Topics:
- Install required packages: mediapipe, opencv-python
- Webcam handling in Colab/local setup

`!pip install mediapipe opencv-python`

------

In [1]:
# Uninstall potentially conflicting libraries and their dependencies
!pip uninstall -y mediapipe opencv-python tensorflow keras pandas numpy

# Reinstall the necessary libraries
!pip install mediapipe opencv-python ipywebrtc

Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━

## ✅ Step 3: Real-Time Hand Tracking Using MediaPipe

**Goal**: Use a webcam feed to track and visualize hand landmarks in real-time.

###Topics:
- Drawing hand landmarks using MediaPipe
- Understanding 3D coordinates of key hand points

In [1]:
# For local runtime
import cv2
import mediapipe as mp
import math

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

print(cap.isOpened())
def get_distance(p1, p2, landmarks):
    x1, y1 = landmarks.landmark[p1].x, landmarks.landmark[p1].y
    x2, y2 = landmarks.landmark[p2].x, landmarks.landmark[p2].y
    return math.hypot(x2 - x1, y2 - y1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)


    cv2.imshow("Hand Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

False


In [2]:
#for web runtime
from google.colab import output
output.enable_custom_widget_manager()

In [7]:
import cv2, mediapipe as mp, math, numpy as np, base64
from IPython.display import display, HTML, clear_output
from google.colab.output import register_callback
import ipywidgets as widgets

# MediaPipe hands
mp_hands = mp.solutions.hands
hands    = mp_hands.Hands(
    min_detection_confidence=0.3,
    min_tracking_confidence=0.3
)
mp_draw = mp.solutions.drawing_utils

# Distance function
def get_distance(p1, p2, landmarks):
    x1, y1 = landmarks.landmark[p1].x, landmarks.landmark[p1].y
    x2, y2 = landmarks.landmark[p2].x, landmarks.landmark[p2].y
    return math.hypot(x2 - x1, y2 - y1)

# Widgets
# Processed frame display
img_widget = widgets.Image(format='jpeg', width=640, height=480)

out        = widgets.Output(layout={'border': '1px solid black'})
display(img_widget, out)

# Python callback for each frame
def process_frame(data_url):
    # Decode JPEG data URL to BGR image
    header, encoded = data_url.split(',', 1)
    frame = cv2.imdecode(
        np.frombuffer(base64.b64decode(encoded), np.uint8),
        cv2.IMREAD_COLOR
    )
    frame = cv2.flip(frame, 1)
    rgb   = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    # Clear and print to the text widget
    with out:
        clear_output(wait=True)
        if not result.multi_hand_landmarks:
            print("No hands detected")
        else:
            for hand_landmarks in result.multi_hand_landmarks:
                # Draw landmarks
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                # Compute & print thumb distance

    # Push updated image to widget
    _, buffer = cv2.imencode('.jpg', frame)
    img_widget.value = buffer.tobytes()

# Register callback
register_callback('notebook.process_frame', process_frame)

# HTML + JavaScript for live capture loop
display(HTML('''
<style>
  #video { border: 2px solid #444; border-radius: 4px; }
  #controls { margin-bottom: 6px; }
</style>
<div id="controls">
  <button id="start">Start</button>
  <button id="stop">Stop</button>
</div>
<video id="video" playsinline autoplay muted width="640" height="480"></video>
<script>
(async function(){
  const video = document.getElementById('video');
  let intervalId;

  // 1) Initialize webcam
  const stream = await navigator.mediaDevices.getUserMedia({
    video: { width: 640, height: 480 }, audio: false
  });
  video.srcObject = stream;

  // 2) Capture & send one frame
  function capture(){
    const canvas = document.createElement('canvas');
    canvas.width  = video.videoWidth;
    canvas.height = video.videoHeight;
    canvas.getContext('2d').drawImage(video, 0, 0);
    const dataURL = canvas.toDataURL('image/jpeg', 0.8);
    google.colab.kernel.invokeFunction(
      'notebook.process_frame',
      [dataURL],
      {}
    );
  }

  // 3) Wire Start/Stop
  document.getElementById('start').onclick = () => {
    if (!intervalId) {
      capture();  // immediate first shot
      intervalId = setInterval(capture, 100);  // ~10 FPS
    }
  };
  document.getElementById('stop').onclick = () => {
    clearInterval(intervalId);
    intervalId = null;
  };
})();
</script>
'''))


Image(value=b'', format='jpeg', height='480', width='640')

Output(layout=Layout(border='1px solid black'))

## ✅ Step 4: Analyze Hand Movement for Cricket Grip (Optional Mini-Project)

**Goal**: Map key finger coordinates and detect specific gestures or grips.

### Topics:
- Detecting custom grip positions using landmark indexes
- Measuring distance/angle between fingers

In [ ]:
import math

def get_distance(p1, p2, landmarks):
    x1, y1 = landmarks.landmark[p1].x, landmarks.landmark[p1].y
    x2, y2 = landmarks.landmark[p2].x, landmarks.landmark[p2].y
    return math.hypot(x2 - x1, y2 - y1)

# inside while loop, after detecting hand_landmarks
thumb_index_dist = get_distance(4, 8, hand_landmarks)
print("Thumb-Index Distance:", thumb_index_dist)

## ✅ Step 5: Creative Application Ideas for Hackathon

- ⚾ **Cricket**: Grip-based swing prediction

- 🏏 **Baseball**: Pitching motion classification

- ⚽ **Football**: Foot-lift tracking using pose estimation

- 🧠 **Rehab**: Range-of-motion analysis post-injury

##✅ Optional Advanced: Using DepthAI or OpenPose (Outside Class Demo)

- [DepthAI Hand Tracker GitHub](https://github.com/geaxgx/depthai_hand_tracker)
- [OpenPose for full-body pose](https://github.com/CMU-Perceptual-Computing-Lab/openpose)

## ✅ Wrap-up & Deliverable Expectations

**Expected Outcomes**:

- A working hand or pose tracking app
- Use of gesture or motion to generate analysis or predictions
- Short video demo or real-time interactive analysis